In [1]:
import pandas as pd
from general_funtions import *
import s3fs 
import json 

In [5]:
config = load_config("config.yaml")

aws_bucket_name = config["aws_s3_connection_info"]["bucket_name"]

data_usa_s3_directory = config["data_usa_connection_info"]["s3_directory"]
data_usa_s3_file_name = config["data_usa_connection_info"]["s3_file_name"]

bls_data_s3_directory = config["bureau_labor_statistics_connection_info"]["s3_directory"]
bls_data_s3_file_name = "pr.data.0.Current"


Get Data USA Data and apply basic data cleansing

In [21]:
data_usa_s3_path = f"s3://{aws_bucket_name}/{data_usa_s3_directory}/{data_usa_s3_file_name}"

fs = s3fs.S3FileSystem()

with fs.open(data_usa_s3_path) as f:
    raw = json.load(f)

df_data_usa = pd.json_normalize(raw["data"])

# Trim whitespace from all column headers
df_data_usa.columns = df_data_usa.columns.str.strip()

# Trim whitespace from all column values
df_data_usa = df_data_usa.applymap(lambda x: x.strip() if isinstance(x, str) else x)

display(df_data_usa)

/var/folders/p4/rt70yb290fs4zwzn3f92_9_r0000gp/T/ipykernel_42288/495264079.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_data_usa = df_data_usa.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


Load BLS Data and apply some basic data cleansing based on data quality issues found when using data downstream in analysis

In [22]:
bls_data_s3_path = f"s3://{aws_bucket_name}/{bls_data_s3_directory}/{bls_data_s3_file_name}"

df_bls_data = pd.read_csv(bls_data_s3_path, sep="\t", compression="gzip", dtype=str)

# Trim whitespace from all column headers
df_bls_data.columns = df_bls_data.columns.str.strip()

# Trim whitespace from all column values
df_bls_data = df_bls_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convert value to numeric for calculations
df_bls_data['value'] = pd.to_numeric(df_bls_data['value'], errors='coerce')

display(df_bls_data)

/var/folders/p4/rt70yb290fs4zwzn3f92_9_r0000gp/T/ipykernel_42288/3089776524.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_bls_data = df_bls_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
37516,PRS88003203,2024,Q04,118.515,NaN
37517,PRS88003203,2024,Q05,118.125,NaN
37518,PRS88003203,2025,Q01,120.226,NaN
37519,PRS88003203,2025,Q02,120.355,NaN


Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [23]:
# Filter df_data_usa for years 2013-2018 and calculate mean and std of Population
df_filtered = df_data_usa[(df_data_usa['Year'] >= 2013) & (df_data_usa['Year'] <= 2018)].copy()
df_filtered['Population'] = pd.to_numeric(df_filtered['Population'])

mean_population = df_filtered['Population'].mean()
std_population = df_filtered['Population'].std()

print(f"Mean Population (2013-2018): {mean_population}")
print(f"Standard Deviation (2013-2018): {std_population}")

Mean Population (2013-2018): 322069808.0
Standard Deviation (2013-2018): 4158441.040908095


Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. 

In [24]:
# Group by series_id and year, sum the values for all quarters
yearly_sums = df_bls_data.groupby(['series_id', 'year'])['value'].sum().reset_index()
yearly_sums.columns = ['series_id', 'year', 'total_value']

# For each series_id, find the year with the maximum total value
best_year_per_series = yearly_sums.loc[yearly_sums.groupby('series_id')['total_value'].idxmax()]

# Rename columns for clarity
best_year_per_series = best_year_per_series.rename(columns={'total_value': 'value'})

# Display the report
display(best_year_per_series)

,series_id,year,value
27,PRS30006011,2022,20.500
58,PRS30006012,2022,17.100
65,PRS30006013,1998,705.895
108,PRS30006021,2010,17.700
139,PRS30006022,2010,12.400
...,...,...,...
8459,PRS88003192,2002,282.800
8512,PRS88003193,2024,862.564
8541,PRS88003201,2022,38.900
8572,PRS88003202,2022,29.700


Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset).

In [25]:
# Filter df_bls_data for the specific series_id and period
filtered_bls = df_bls_data[(df_bls_data['series_id'] == 'PRS30006032') & (df_bls_data['period'] == 'Q01')].copy()

# Convert year to numeric for merging
filtered_bls['year'] = pd.to_numeric(filtered_bls['year'], errors='coerce')

# Merge with population data
report = filtered_bls.merge(df_data_usa[['Year', 'Population']], 
                            left_on='year', 
                            right_on='Year', 
                            how='inner')

# Drop duplicate Year column and display
report = report.drop(['Year', 'footnote_codes'], axis=1)

display(report)

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,0.5,316128839.0
1,PRS30006032,2014,Q01,-0.1,318857056.0
2,PRS30006032,2015,Q01,-1.7,321418821.0
3,PRS30006032,2016,Q01,-1.4,323127515.0
4,PRS30006032,2017,Q01,0.9,325719178.0
5,PRS30006032,2018,Q01,0.5,327167439.0
6,PRS30006032,2019,Q01,-1.6,328239523.0
7,PRS30006032,2021,Q01,0.7,331893745.0
8,PRS30006032,2022,Q01,5.3,333287562.0
9,PRS30006032,2023,Q01,0.3,334914896.0
